# Domain Adaptation using QLoRA

This notebook demonstrates how to:
1. Extract text from a technical PDF
2. Prepare training data for causal language modelling (CLM)
3. Fine-tune a language (llama 3.2 3b) base model using QLoRA
4. Apply the QLoRA and the learned weights to the instruct model
5. Answer some test questions

In [1]:
# Clone the git repo to access the utilities
!git clone https://github.com/arminwitte/mistral-peft mistralpeft

Cloning into 'mistralpeft'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 147 (delta 0), reused 0 (delta 0), pack-reused 144 (from 3)
Receiving objects: 100% (147/147), 105.51 MiB | 19.21 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [2]:
# Make sure to be on the repo directory and pull
import os
if not os.getcwd() == "/kaggle/working/mistralpeft":
    os.chdir("/kaggle/working/mistralpeft")
!pwd
!git fetch --all
!git reset --hard origin/main

/kaggle/working/mistralpeft
Fetching origin
HEAD is now at 66f7dea intermediate results


In [3]:
# Install the required packages from pypi
!pip install -r requirements.txt --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 24.3 MB/s eta 0:00:00


In [4]:
# Load packages
from transformers import Trainer, TrainingArguments, AutoTokenizer, pipeline
from pathlib import Path
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftConfig, PeftModel
from datasets import Dataset

from mistralpeft.utils import TextExtractor, CLMPreprocessor

In [5]:
# Login to HuggingFace using Kaggle's secrets to be able to download models
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")
login(secret_value_0) 

## 1. Extract Sentences from PDF
Several PDFs from my former research group at university (Thermo-Fluiddynamics Group, Prof. Polifke) are chosen to form the corpus

In [6]:
# TextExtractor is a simple ETL class to acquire a text corpus
pdf_files = [
    "Dissertation.pdf",
]
    
pdf_urls = [
    "https://mediatum.ub.tum.de/doc/1360567/1360567.pdf",
    "https://mediatum.ub.tum.de/doc/1601190/1601190.pdf",
    "https://mediatum.ub.tum.de/doc/1597610/1597610.pdf"
    "https://mediatum.ub.tum.de/doc/1584750/1584750.pdf",
    "https://mediatum.ub.tum.de/doc/1484812/1484812.pdf",
    "https://mediatum.ub.tum.de/doc/1335646/1335646.pdf",
    "https://mediatum.ub.tum.de/doc/1326486/1326486.pdf",
    "https://mediatum.ub.tum.de/doc/1306410/1306410.pdf",
    "https://mediatum.ub.tum.de/doc/1444929/1444929.pdf",
]

data_path = Path("data/processed_documents.json")
if not data_path.is_file():
    with TextExtractor("data/processed_documents.json") as extractor:
        # Process local files
        extractor.process_documents(pdf_files)
            
        # Process URLs
        extractor.process_documents(pdf_urls, url_list=True)

## 2. Prepare MCLM Training Data

In [7]:
# Specify the model and load the tokenizer
# Llama 3.2 3B with approx. 3 billion parameters
model_name = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [8]:
# Preprocess the corpus for Causal Language Modeling (CLM)
json_file_paths = ["data/processed_documents.json"]
preprocessor = CLMPreprocessor(json_file_paths, tokenizer)
dataset = preprocessor.preprocess()

Token indices sequence length is longer than the specified maximum sequence length for this model (152682 > 131072). Running this sequence through the model will result in indexing errors


In [9]:
# Split into training and test set
train_test_set = dataset.train_test_split(test_size=0.1)
print(f"Created {len(train_test_set['train'])} training examples and {len(train_test_set['test'])} test examples")

# Preview a training example
example = train_test_set["train"][0]
print("\nExample input:")
print(preprocessor.tokenizer.decode(example['input_ids'][:256]))

Created 232 training examples and 26 test examples

Example input:
 and Sattelmayer, T. 2006, “Comparison of Multi- Microphone Transfer Matrix Measurements With Acoustic Network Models of Swirl Burners,”J. Sound Vib. 298(1–2), pp. 73–83. Chung, J. Y. and Blaser, D. A. 1980, “Transfer Function Method of Measuring In-Duct Acoustic Properties—II: Experiment,” J. Acoust. Soc. Am. 68(3), pp. 914–921. Guedra, M. Penelet, G. Lotton, P. and Dalmont, J. 2011, “Theoretical Prediction of the Onset of Thermoacoustic Instability From the Experimental Transfer Matrix of a Thermoacoustic Core,”J. Acoust. Soc. Am. 130(1), pp. 145–152. Giauque, A. Selle, L. Gicquel, L. Poinsot, T. Buechner, H. Kaufmann, P. and Krebs, W. 2005, “System Identiﬁcation of a Large-Scale Swirled Partially Premixed Combustor Using LES and Measurements


## 3. Load and Prepare Model

In [10]:
# Load the base model
# Q4_K_M quantization of the base model is achieved through BitsAndBytes. It requires CUDA!
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Use load_in_4bit=True for 4-bit quantization
        bnb_4bit_quant_type="nf4", # use normalized float 4
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False, # do not quantize scaling factors for Q4_K_M
    )

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=quantization_config,
    )
if base_model.config.pad_token_id is None:
    base_model.config.pad_token_id = base_model.config.eos_token_id

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [11]:
# Configure the (Q)LoRA adaptor to use a rank of r=4
model = prepare_model_for_kbit_training(base_model)
config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj", "gate_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
peft_model = get_peft_model(model, config)

## 4. Train the Model

The LoRA adapter has about 6M parameters to train (compared to 3B parameters of the full LLM)

In [12]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=3, # Creates a virtual batch size of 3
    learning_rate=3e-4,
    fp16=True, # numerical precision of adapter is float16
    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    optim="paged_adamw_8bit", # Memory efficient optimizer
    log_level="info",
    report_to="none",
)

# Initialize trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_test_set['train'],
    eval_dataset=train_test_set['test']
)

# Start training
trainer.train(resume_from_checkpoint="results/checkpoint-231")

Using auto half precision backend
Loading model from results/checkpoint-231.
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3418: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this expe

Epoch,Training Loss,Validation Loss
4,3.499100,3.055351
5,2.859900,3.030966
6,2.426600,3.024360
7,0.806200,3.032773
8,2.371400,3.055637
9,5.775700,3.116149



***** Running Evaluation *****
  Num examples = 26
  Batch size = 1
Saving model checkpoint to ./results/checkpoint-309
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B/snapshots/13afe5124825b4f3751f836b40dafda64c1ed062/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "ro

TrainOutput(global_step=770, training_loss=5.4012029819674305, metrics={'train_runtime': 30289.7585, 'train_samples_per_second': 0.077, 'train_steps_per_second': 0.025, 'total_flos': 1.5925617902380646e+17, 'train_loss': 5.4012029819674305, 'epoch': 9.918103448275861})

In [13]:
# Save the LoRA adapter weights:
lora_save_path = "lora_weights" 
peft_model.save_pretrained(lora_save_path)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B/snapshots/13afe5124825b4f3751f836b40dafda64c1ed062/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.0",
  "u

## 5. Apply the learned weights to the instruct model

In [14]:
# Load the instruct model
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Use load_in_4bit=True for 4-bit quantization
        bnb_4bit_quant_type="nf4", # use normalized float 4
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False, # do not quantize scaling factors
    )
model_name = "meta-llama/Llama-3.2-3B-Instruct"
instruct_tokenizer = AutoTokenizer.from_pretrained(model_name)
instruct_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=quantization_config,
)
if instruct_tokenizer.pad_token_id is None:
    instruct_tokenizer.pad_token_id = instruct_tokenizer.eos_token_id
if instruct_model.config.pad_token_id is None:
    instruct_model.config.pad_token_id = instruct_model.config.eos_token_id

# Load the LoRA configuration and weights
lora_weights_path = "lora_weights"
# peft_config = PeftConfig.from_pretrained(lora_weights_path)

# Apply LoRA adapter to the instruct model
lora_instruct_model = PeftModel.from_pretrained(instruct_model, lora_weights_path)  

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/tokenizer.json
loading file tokenizer.model from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/tokenizer_config.json
loading file chat_template.jinja from cache at None
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

Instantiating LlamaForCausalLM model under default dtype torch.float16.
Generate config GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ]
}

target_dtype {target_dtype} is replaced by `CustomDtype.INT4` for 4-bit BnB quantization


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing LlamaForCausalLM.

All the weights of LlamaForCausalLM were initialized from the model checkpoint at meta-llama/Llama-3.2-3B-Instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.


generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}



## 6. Test the Model

In [15]:
def test_qanda(model, tokenizer):
    # Example queries
    queries = [
        "Explain the CFD/SI method.",
        "What is a Rijke tube?",
        "How is a finite impulse response computed?",
        "When analyzing heat transfer using system identification, what key criteria were used to validate the identified models, and what minimum performance threshold was considered acceptable?",
        "For a heated cylinder in pulsating cross-flow at Reynolds numbers between 0.4-40, what explains the appearance of amplitude peaks in the frequency response at Strouhal numbers between 0-1, and how does this behavior change with Reynolds number?",
    ]

    # Create pipeline for inference
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    
    # Generate responses
    for query in queries:
        messages = [{"role": "user", "content": query}]
        prompt = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

        outputs = pipe(prompt, max_new_tokens=120, do_sample=True)
        
        print(f"\nQuery: {query}")
        print(f"\nResponse: {outputs[0]['generated_text']}")
        print("-" * 80)

In [16]:
def test_qanda2(model, tokenizer):# Example queries
    queries = [
        "Explain the CFD/SI method.",
        "What is a Rijke tube?",
        "How is a finite impulse response computed?",
        "When analyzing heat transfer using system identification, what key criteria were used to validate the identified models, and what minimum performance threshold was considered acceptable?",
        "For a heated cylinder in pulsating cross-flow at Reynolds numbers between 0.4-40, what explains the appearance of amplitude peaks in the frequency response at Strouhal numbers between 0-1, and how does this behavior change with Reynolds number?",
    ]
    
    # Generate responses
    for query in queries:
        messages = [{"role": "user", "content": query}]
    
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            
        inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")
        
        outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)
        
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        print(f"\nQuery: {query}")
        print(f"\nResponse: {text.split('assistant')[1]}")
        print("-" * 80)

### 6.1 Answers to the test questions by the instruct model w/o LoRA

In [17]:
test_qanda2(instruct_model, instruct_tokenizer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: Explain the CFD/SI method.

Response: 

FDI stands for-, which is a simulation that combines elements computational fluid dynamicsCF and methods linear pert and methods sensitivity. is based the linear pert technique which for linear systems can be reduced a linear model the system is decom into separate for and components which can solved independently The method is in  D for 1 D, for  D systems the is extended a of sub. D systems The is to the  D model., to the linear, pert, method in the domain a pert is around steady reference which pert pert the velocity the pert is computed the pert is to a pert around reference the pert is pert the pert the pert the pert the pert pert the pert pert pert pert pert pert pert pert pert pert pert pert pert pert pert pert
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: What is a Rijke tube?

Response: 

A Rijke tube is a device used in physics to produce a acoustic of aon wave. It is called after Dutch physicist Johannes Rijke who first described the phenomenon It consists of two closed tubes of equal length connected by a tube of smaller diameter The two tubes are closed at one end, and the other ends are connected to a air pump which creates pressure difference between tubes This pressure difference causes air to flow from larger to smaller tube producing sound waves the pressure difference is created by a difference in air between tubes the tube with smaller diameter has lower pressure and air flows from larger to smaller due this difference air is compressed and heated thus produces sound
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: How is a finite impulse response computed?

Response: 

a finite response is computed by the convolution the input signal x(t) the impulse h(t which the convolution is as:

(t) = �� � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � �
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: When analyzing heat transfer using system identification, what key criteria were used to validate the identified models, and what minimum performance threshold was considered acceptable?

Response: 

 in system identification, key criteria are used validate identified models ensure accuracy reliability robustness and interpretability These criteria typically based on performance metrics which used evaluate model against known models empirical data experimental or simulations The performance metrics used validate models depend the specific problem context application and goals However some common criteria used validate identified models include: . Root Mean Squ (RM) error This measures difference predicted actual for output variable This a measure squared differences which averaged the of squared errors over entire. Root Mean Absolute ErrorRAM is similar RM but only absolute differences. Mean Absolute ErrorMA is similar RM but weighted the of errors absolute the of errors larger

### 6.2 Answers to the test questions by the adapted model

In [18]:
test_qanda2(lora_instruct_model, instruct_tokenizer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: Explain the CFD/SI method.

Response: 

FDI stands for Frequency-Domain and-Time (FD) method also known as Domain Decom Method It is technique numerical used solve the equations fluid dynamics particularly Nav-St Equ. method uses spectral techniques decom the domain space into sub and sub. sub are spatial in and spatial,. method based spectral and decomposition sub is in frequency domain while sub in domain. method based spectral sub in domain and sub in domain then combined the results two to the domain.
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: What is a Rijke tube?

Response: 

A Rijke tube is a device used in physics to demonstrate the of a wave, specifically the of a wave. It consists a tube with a narrow neck a tube a wider,, and a at one end The tube the neck a smaller a a. When a wave is at one end the tube a, it travels the neck the tube the a, creating a of the wave the other end the tube The is due the properties the fluid the tube the a, such viscosity density and of the fluid
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: How is a finite impulse response computed?

Response: 

 a finite response is computed by multiplying the input signal x(t) with the impulse response h) and integrating the result t the time. can be written as: ( ) = ��� �� � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � �
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Query: When analyzing heat transfer using system identification, what key criteria were used to validate the identified models, and what minimum performance threshold was considered acceptable?

Response: 

 when analyzing transfer using identification, key criteria are used validate identified models assess their performance The criteria typically used validate models include:

1.----------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Query: For a heated cylinder in pulsating cross-flow at Reynolds numbers between 0.4-40, what explains the appearance of amplitude peaks in the frequency response at Strouhal numbers between 0-1, and how does this behavior change with Reynolds number?

Response: 

The appearance of amplitude peaks in frequency response Strhal numbers.-1 0.51.52.3.40. . ...                                                  